# Helper

In [171]:
import math

class Level:
    def __init__(self):
        return
    
    def emptyInit(self,width,height):
        self.width=width
        self.height=height
        
        self.map=[]
        for y in range(self.height):
            self.map.append([])
            for x in range(self.width):
                self.map[y].append(" ")
        
    def stringInit(self,level):
        lines=level.split("\n")
        self.width=0
        for i in range(len(lines)):
            if len(lines[i].strip()) == 0:
                del lines[i]
                i-=1
            if len(lines[i]) > self.width:
                self.width=len(lines[i])
        self.height=len(lines)
        self.map=[]
        for y in range(self.height):
            self.map.append([])
            for x in range(self.width):
                if x < len(lines[y]):
                    self.map[y].append(lines[y][x])
                else:
                    self.map[y].append(" ")
                    
    def getTileNumber(self, tiles):
        if not isinstance(tiles, list):
            tiles=[tiles]
        result = 0
        for y in range(self.height):
            for x in range(self.width):
                if self.map[y][x] in tiles:
                    result += 1
        return result
    
    def __str__(self):
        result=""
        for y in range(self.height):
            for x in range(self.width):
                result+=self.map[y][x]
            result+="\n"
        return result[:-1]
    
    def rotate(self):
        result=Level()
        result.emptyInit(self.height, self.width)
        for y in range(self.height):
            for x in range(self.width):
                newX=y
                newY=x
                result.map[newY][newX]=self.map[y][x]
        return result
    
    def reflect(self, horizontal=False, vertical=False):
        result=Level()
        result.emptyInit(self.width, self.height)
        for y in range(self.height):
            for x in range(self.width):
                newX=x
                if horizontal:
                    newX=self.width-x-1
                newY=y
                if vertical:
                    newY=self.height-y-1
                result.map[newY][newX]=self.map[y][x]
                result.map[y][x]=self.map[newY][newX]
        return result
    
    def maxTranslate(self, maxWidth, maxHeight):
        return maxWidth - self.width, maxHeight - self.height
    
    def translate(self,sX,sY,maxWidth,maxHeight,padding="#"):
        result=Level()
        result.emptyInit(maxWidth, maxHeight)
        for y in range(maxHeight):
            if y < sY or y > sY+self.height-1:
                for x in range(maxWidth):
                    result.map[y][x] = padding
            else:
                start=-1
                end=-1
                for x in range(self.width):
                    if self.map[y-sY][x] == "#" and start==-1:
                        start=x
                    if self.map[y-sY][self.width-1-x] == "#" and end == -1:
                        end = self.width-1-x
                for x in range(maxWidth):
                    if x < sX+start or x > sX+end:
                        result.map[y][x] = padding
                        continue
                    result.map[y][x] = self.map[y-sY][x-sX]
        return result

# Split Level

In [172]:
gameLevels=[]
index=0
for i in range(4):
    f=open("OriginalData/Microban_" + str(i+1) + ".txt")
    lines=f.readlines()
    f.close()
    
    level=""
    for l in lines:
        if len(l.strip()) == 0 or ";" in l:
            if len(level) > 0:
                f=open("SplitData/Level_"+str(index)+".txt", "w")
                f.write(level)
                f.close()
                index+=1
                temp=Level()
                temp.stringInit(level)
                gameLevels.append(temp)
            level=""
            continue
        level += l

# Level Statistics

In [177]:
maxSize=10

histogram=[0]*50
for l in gameLevels:
    if max(l.width,l.height) > maxSize and maxSize > 0:
        continue
    histogram[max(l.width,l.height)] += 1
print("LevelSize")   
print(histogram)

histogram=[0]*50
for l in gameLevels:
    if max(l.width,l.height) > maxSize and maxSize > 0:
        continue
    if l.getTileNumber(["$", "*"])==16:
        print(l)
    histogram[l.getTileNumber(["$", "*"])] += 1
print("CrateNumbers")
print(histogram)

LevelSize
[0, 0, 0, 0, 0, 1, 1, 31, 75, 85, 69, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#########
#       #
# *$*$* #
# $...$ #
# *. .* #
# $...$ #
# *$*$* #
#      @#
#########
CrateNumbers
[0, 1, 31, 104, 62, 18, 20, 4, 7, 1, 2, 1, 10, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Generate Dataset

In [159]:
import os

outputPath = 'OutputData/'
maxSize=10
index=0

def writeLevel(index, level):
    f=open(outputPath + "level_" + str(index)+".txt", "w")
    f.write(str(level))
    f.close()

for old_level in os.listdir(outputPath):
    os.unlink(os.path.join(outputPath, old_level))
    
for l in gameLevels:
    if max(l.width, l.height) > maxSize:
        continue
    levels=[l.reflect(False,False),l.reflect(True,False),
            l.reflect(False,True),l.reflect(True,True),
            l.rotate().reflect(False,False), l.rotate().reflect(True,False), 
            l.rotate().reflect(False,True), l.rotate().reflect(True,True)]
    for tl in levels:
        mx,my=tl.maxTranslate(maxSize,maxSize)
        for x in range(0,mx+1):
            for y in range(0, my+1):
                writeLevel(index,tl.translate(x,y,maxSize,maxSize,"#"))
                index+=1
#                 writeLevel(index,tl.translate(x,y,maxSize,maxSize," "))
#                 index+=1
#                 writeLevel(index,tl.translate(x,y,maxSize,maxSize,"*"))
#                 index+=1